In [1]:
import pandas as pd

In [2]:
df_PlayTimeGenre = pd.read_csv("funciones/PlayTimeGenre.csv", low_memory=False)

In [7]:
def PlayTimeGenre(genre: str):
    
    genre = genre.capitalize()

    genero_elegido = df_PlayTimeGenre[df_PlayTimeGenre['genre'] == genre]

    if genero_elegido.empty:
        return {"No hay información para el género": genre}
    
    anio = genero_elegido['release_date'].values[0]

    return {"El año con más horas jugadas para el género " + genre + " es el " + str(anio)}

In [8]:
PlayTimeGenre("acTion")

{'El año con más horas jugadas para el género Action es el 2012'}

In [10]:
df_UserForGenre_1 = pd.read_csv("funciones/UserForGenre_1.csv", low_memory=False)
df_UserForGenre_2 = pd.read_csv("funciones/UserForGenre_2.csv", low_memory=False)

In [9]:
def UserForGenre(genre: str = None):
    
    genre = genre.capitalize()

    genero_elegido = df_UserForGenre_1[df_UserForGenre_1['genre'] == genre]

    if genero_elegido.empty:
        return {"No hay información para el género": genre}

    usuario = genero_elegido['user_id'].values[0]

    usuario_top = df_UserForGenre_2[df_UserForGenre_2['user_id'] == usuario]

    lista_horas_jugadas = [
    {"Año": anio, "minutos": tiempo} for anio, tiempo in zip(usuario_top['posted'], usuario_top['playtime_forever'])
    ]

    return {
        "Usuario": "El usuario con más horas jugadas para el género " + genre + " es " + usuario,
        "Horas jugadas": lista_horas_jugadas
    }

In [11]:
UserForGenre("action")

{'Usuario': 'El usuario con más horas jugadas para el género Action es Sp3ctre',
 'Horas jugadas': [{'Año': '2015', 'minutos': 318642},
  {'Año': 'sin dato de año', 'minutos': 1380665}]}

In [8]:
df_UsersRecommend = pd.read_csv("funciones/UsersRecommend.csv", low_memory=False)

In [13]:
def UsersRecommend(posted: int):

    df_anio = df_UsersRecommend[df_UsersRecommend['posted'] == posted]

    top_3_dict = {}
    for i, row in df_anio.iterrows():
        puesto = "Puesto " + str(i + 1 - df_anio.index[0])
        top_3_dict[puesto] = row['app_name']

    return top_3_dict

In [15]:
UsersRecommend(2015)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': "Garry's Mod"}

In [10]:
df_UsersNotRecommend = pd.read_csv("funciones/UsersNotRecommend.csv", low_memory=False)

In [11]:
def UsersNotRecommend(posted: int):

    df_anio = df_UsersNotRecommend[df_UsersNotRecommend['posted'] == posted]

    top_3_dict = {}
    for i, row in df_anio.iterrows():
        puesto = "Puesto " + str(i + 1 - df_anio.index[0])
        top_3_dict[puesto] = row['app_name']

    return top_3_dict

In [12]:
UsersNotRecommend(2015)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'DayZ',
 'Puesto 3': 'PAYDAY 2'}

In [3]:
df_sentiment_analysis = pd.read_csv("funciones/sentiment_analysis.csv", low_memory=False)

In [7]:
def Sentiment_Analysis(year: int):
    # Filtrar el DataFrame para obtener las filas que coinciden con el release_date dado
    df_anio = df_sentiment_analysis[df_sentiment_analysis['release_date'] == year]
    
    # Convertir las filas coincidentes a una lista de diccionarios
    filas_lista = df_anio.to_dict(orient='records')
    
    return filas_lista

In [9]:
Sentiment_Analysis(2000)

[{'release_date': 2000, 'Negativo': 18, 'Neutro': 32, 'Positivo': 49}]